In [1]:
from spacy.lang.en import English
from tqdm.notebook import tqdm

nlp = English()
# nlp=spacy.load('en_core_web_lg')
nlp.add_pipe("sentencizer")
doc = nlp("This is a sentence.  Dr. ___ thought this. upper lobe 2.5 cm -- ? aspiration. Talked to at 9:35 a.m. 1. abc. 2. def.")
print(list(doc.sents))

[This is a sentence.,  Dr. ___ thought this., upper lobe 2.5 cm -- ?, aspiration., Talked to at 9:35 a.m. 1., abc., 2., def.]


In [2]:
# import pandas
# df = pandas.read_csv("../../mimic_unique.csv", delimiter=",")

In [3]:
# print(df.head())
# print(len(df))
# # print(df.count())

In [2]:
def split(val):
    #old_idx, fname, rep = val[0], val[1], val[2]
    fname, rep = val[0], val[1]
    sentences = [str(i) for i in list(nlp(rep).sents)]
    pruned = [i.replace("\"", "") for i in sentences if '_' not in i and len(i) > 5]
    return fname, pruned

In [5]:
# newdf = pandas.DataFrame(columns=["Old Index", "filename", "Report Impression Sentence"])
# unique_sents = set()
# counter = 0
# for i in tqdm(df.itertuples(), total=len(df)):
#     idx, old_idx, fname, rep = i[0], i[1], i[2], i[3]
#     sentences = list(nlp(rep).sents)
#     for sent in sentences:
#         unique_sents.add(str(sent))
#         row = {'Old Index': old_idx, 'filename': fname, 'Report Impression Sentence': str(sent)}
#         newdf = newdf.append(row, ignore_index = True)
#     if counter % 10000 == 0:
#         newdf.to_csv("sentences.csv", index=False)
#     counter += 1

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
sc = SparkContext(conf=SparkConf())
spark = SparkSession \
    .builder \
    .appName("sentence segmentation") \
    .getOrCreate()

In [4]:
#sdf = spark.read.csv("../../mimic_unique.csv", header=True)
sdf = spark.read.csv("/deep/group/data/med-data/mimic_train_findings.csv", header=True)
#sdf = sdf.withColumnRenamed('_c0', 'index')
print(sdf.head())
print(sdf.count())

Row(filename='s57146986.txt', impression='   As compared to the previous radiograph, there is no relevant change.  Normal size of the cardiac silhouette.  No pulmonary edema.  No pleural effusions.  No focal parenchymal opacity suggesting pneumonia.  Normal appearance of the hilar and mediastinal structures.  IMPRESSION:   No acute cardiopulmonary process.')
320027


In [5]:
from pyspark.sql.functions import explode
sents = sdf.rdd.map(lambda x: split(x))
#new_sent_df = spark.createDataFrame(data=sents, schema = ['index','filename','sentence'])
new_sent_df = spark.createDataFrame(data=sents, schema = ['filename','sentence'])
#new_sent_df = new_sent_df.select(new_sent_df.index, new_sent_df.filename, explode(new_sent_df.sentence))
new_sent_df = new_sent_df.select(new_sent_df.filename, explode(new_sent_df.sentence))

In [6]:
print(new_sent_df.take(10))

[Row(filename='s57146986.txt', col='   As compared to the previous radiograph, there is no relevant change.'), Row(filename='s57146986.txt', col=' Normal size of the cardiac silhouette.'), Row(filename='s57146986.txt', col=' No pulmonary edema.'), Row(filename='s57146986.txt', col=' No pleural effusions.'), Row(filename='s57146986.txt', col=' No focal parenchymal opacity suggesting pneumonia.'), Row(filename='s57146986.txt', col=' Normal appearance of the hilar and mediastinal structures.'), Row(filename='s57146986.txt', col=' IMPRESSION:   No acute cardiopulmonary process.'), Row(filename='s57146986.txt', col='   As compared to the previous radiograph, there is no relevant change.'), Row(filename='s57146986.txt', col=' Normal size of the cardiac silhouette.'), Row(filename='s57146986.txt', col=' No pulmonary edema.')]


In [7]:
new_sent_df.coalesce(1).write.csv("sentence_dir")

In [8]:
print(new_sent_df.take(10))

[Row(filename='s57146986.txt', col='   As compared to the previous radiograph, there is no relevant change.'), Row(filename='s57146986.txt', col=' Normal size of the cardiac silhouette.'), Row(filename='s57146986.txt', col=' No pulmonary edema.'), Row(filename='s57146986.txt', col=' No pleural effusions.'), Row(filename='s57146986.txt', col=' No focal parenchymal opacity suggesting pneumonia.'), Row(filename='s57146986.txt', col=' Normal appearance of the hilar and mediastinal structures.'), Row(filename='s57146986.txt', col=' IMPRESSION:   No acute cardiopulmonary process.'), Row(filename='s57146986.txt', col='   As compared to the previous radiograph, there is no relevant change.'), Row(filename='s57146986.txt', col=' Normal size of the cardiac silhouette.'), Row(filename='s57146986.txt', col=' No pulmonary edema.')]


In [58]:
unique_sents = set()
# newdf = pandas.DataFrame(columns=["index", "filename", "Report Impression Sentence"])
# tempo_df = sdf.rdd.map(lambda x: split(x)).collect()
for val in tqdm(tempo_df, total=len(tempo_df)):
    idx, fname, sentences = val
    unique_sents = unique_sents | set(sentences)
#     for sentence in sentences:
#         unique_sents.add(str(sentence))
#         row = {'index': idx, 'filename': fname, 'Report Impression Sentence': sentence}
#         newdf = newdf.append(row, ignore_index = True)
# newdf.to_csv("sentences.csv", index=False)
print(len(unique_sents))


264000


In [68]:
temp = sorted(list(unique_sents), key=lambda x: len(x))
print(temp[:500])

['s', 'S', '?', '.', 'I', 'A', '6', 'He', 'PE', 'No', 'An', 'hh', 'Or', '3.', 'It', 'AM', '1.', '4.', 'pm', 'MD', '2.', 'Dx', '6.', "'s", 'am', '2)', 'NG', 'E.', '5.', '7.', 'Is', 'Ill', 'CHF', 'At.', 'The', 'It.', 'All', 'New', '"1.', 'And', 'No.', 'He.', 'am.', 'pm.', '___', 'End', 'Are', 'Ref.', 'One.', 'The.', '2. ?', 'ETT.', 'CHF.', '3. ?', '4) ?', '___,', 'Mild', 'What', 'A.m.', 'Item', 'COPD', '___.', '2) ?', 'Were', 'P.m.', 'Stop', '3) ?', 'Left', '4.___', 'COPD.', 'UZRD.', 'None.', 'bulla', 'P ___', 'Mass.', 'Whole', 'Zones', 'Ileus', 'Sixth', 'sixth', 'Edema.', 'Ileus.', 'Goiter', 'Above.', 'Insert', 'Would.', 'Drain.', '2. ___', 'Essed.', '1. ___', 'ASCVD.', 'Venous', 'Per CT', 'Slight', '12:31.', 'Signed', 'GOITER', 'No TB.', 'Should', '4. ___', 'Normal', 'Number', '3. ___', 'Little', 'Errant', 'Its is', 'COPD 2.', 'Stomach', 'Stable.', 'MD, MPH', '3-4 cm.', 'However', 'ET tube', 'Normal.', 'on ___.', 'No left', '2) CHF.', 'No CHF.', '2. ___%', 'All all', 'Film 2.', 'COPD 3